# Q1. Running Elastics

In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
print(es.info())

{'name': '5abbf0fa4a10', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EodWOTaZS_GZ-ci2XIN9sg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [2]:
es.info()['version']['build_hash']

'42f05b9372a9a4a470db3b52817899b99a76ee73'

# Q2. Indexing the data

In [3]:
import requests

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)


In [4]:
documents[-10:]

[{'text': 'Problem description\nFollowing video 6.3, at minute 11:23, get records command returns empty Records.\nSolution description\nAdd --no-sign-request to Kinesis get records call:\n aws --endpoint-url=http://localhost:4566 kinesis get-records --shard-iterator […] --no-sign-request',
  'section': 'Module 6: Best practices',
  'question': 'Empty Records in Kinesis Get Records with LocalStack',
  'course': 'mlops-zoomcamp'},
 {'text': "Problem description\ngit commit -m 'Updated xxxxxx'\nAn error has occurred: InvalidConfigError:\n==> File .pre-commit-config.yaml\n=====> 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte\nSolution description\nSet uft-8 encoding when creating the pre-commit yaml file:\npre-commit sample-config | out-file .pre-commit-config.yaml -encoding utf8\nAdded by MarcosMJD",
  'section': 'Module 6: Best practices',
  'question': 'In Powershell, Git commit raises utf-8 encoding error after creating pre-commit yaml file',
  'course': 'mlops-

In [11]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type" : "keyword"}
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [12]:
from tqdm.notebook import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

# Q3. Searching

In [7]:
user_question = "How do I execute a command in a running docker container?"

index_name = "course-questions"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text", "section"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [8]:
response = es.search(index=index_name, body=search_query)

In [9]:
print(response['hits']['hits'][0]['_score'])

84.050095


# Q4. Filtering
**Limit** the questions to `machine-learning-zoomcamp`.  
Return **3 results**

In [11]:
from elasticsearch import Elasticsearch, ApiError

# Initialize Elasticsearch client
es = Elasticsearch("http://localhost:9200")

def retrieve_documents(user_question, index_name="course-questions"):
    # Define the search query
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": user_question,
                        "fields": ["question^4", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    try:
        # Execute the search query
        response = es.search(
            index=index_name,
            body=search_query
        )
        return [hit for hit in response['hits']['hits']]
    except ApiError as e:
        print(f"Error executing search query: {e}")
        return []
    

In [12]:
user_question = "How do I execute a command in a running docker container?"
results = retrieve_documents(user_question)
for result in results:
    print(result, end="\n\n")

{'_index': 'course-questions', '_id': 'QGcoOpABdJaMY-5OYnYr', '_score': 84.050095, '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)', 'section': '5. Deploying Machine Learning Models', 'question': 'How do I debug a docker container?', 'course': 'machine-learning-zoomcamp'}}

{'_index': 'course-questions', '_id': 'X2coOpABdJaMY-5OY3Yw', '_score': 51.04628, '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_

In [13]:
for result in results:
    question = result['_source']['question']
    print(f"Question: {question}", end="\n\n")

Question: How do I debug a docker container?

Question: How do I copy files from my local machine to docker container?

Question: How do I copy files from a different folder into docker container’s working directory?



# Q5. Building a Prompt

In [16]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [17]:
context_entries = []
for result in results:
    question = result['_source']['question']
    text = result['_source']['text']
    context = context_template.format(question=question, text=text)
    context_entries.append(context)

contexts = '\n\n'.join(context_entries)
print(contexts)

Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:/path/in/container
Hrithik Kumar Advani

Q: How do I copy files from a different folder into docker container’s working directory?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to 

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [19]:
question = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question=question, context=contexts)

In [20]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [21]:
len(prompt)

1462

# Q6. Tokens

In [22]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o")

In [24]:
tokens = encoding.encode(prompt)

In [25]:
len(tokens)

322

# Bonus: Generationg the answer

In [26]:
from dotenv import load_dotenv

load_dotenv()

True

In [28]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[
        {'role': 'user', 'content': prompt}
    ]
)

In [29]:
response

ChatCompletion(id='chatcmpl-9dJTn50Kzl7RglfmOJDQu0S1zjTzo', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container by using the `docker ps` command. Then, use `docker exec -it` followed by the container ID and the command you want to execute. For example, to start a bash session in the running container, you would use the following commands:\n\n1. Find the container ID:\n   ```bash\n   docker ps\n   ```\n\n2. Execute the bash command in the specific running container:\n   ```bash\n   docker exec -it <container-id> bash\n   ```', role='assistant', function_call=None, tool_calls=None))], created=1719156699, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_3e7d703517', usage=CompletionUsage(completion_tokens=128, prompt_tokens=329, total_tokens=457))

In [35]:
print(response.to_json())

{
  "id": "chatcmpl-9dJTn50Kzl7RglfmOJDQu0S1zjTzo",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container by using the `docker ps` command. Then, use `docker exec -it` followed by the container ID and the command you want to execute. For example, to start a bash session in the running container, you would use the following commands:\n\n1. Find the container ID:\n   ```bash\n   docker ps\n   ```\n\n2. Execute the bash command in the specific running container:\n   ```bash\n   docker exec -it <container-id> bash\n   ```",
        "role": "assistant"
      }
    }
  ],
  "created": 1719156699,
  "model": "gpt-4o-2024-05-13",
  "object": "chat.completion",
  "system_fingerprint": "fp_3e7d703517",
  "usage": {
    "completion_tokens": 128,
    "prompt_tokens": 329,
    "

In [34]:
print(response.choices[0].message.content)

To execute a command in a running Docker container, you can use the `docker exec` command. First, find the container ID of the running container by using the `docker ps` command. Then, use `docker exec -it` followed by the container ID and the command you want to execute. For example, to start a bash session in the running container, you would use the following commands:

1. Find the container ID:
   ```bash
   docker ps
   ```

2. Execute the bash command in the specific running container:
   ```bash
   docker exec -it <container-id> bash
   ```


In [36]:
response.usage

CompletionUsage(completion_tokens=128, prompt_tokens=329, total_tokens=457)

# Bonus: calculating the cost

In [37]:
# for gpt-4o, in dolars per 1M tokens
input_cost = 5
output_cost = 15

def calculate_cost(input_tokens, output_tokens):
    total_cost = input_tokens * input_cost / 10e6 + output_tokens * output_cost / 10e6
    return total_cost

input_tokens = response.usage.prompt_tokens
output_tokens = response.usage.completion_tokens

cost_per_request = calculate_cost(input_tokens, output_tokens)

num_requests = 1000

total_cost = cost_per_request * num_requests
print(f"Total cost for {num_requests} requests: ${total_cost:.2f}")

Total cost for 1000 requests: $0.36
